# 1. PREPROCESSING

# Imports libraries

In [97]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer #process null values
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import lightgbm as lgb
# system
import itertools
# matplotlib and seaborn for plotting
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sns

# Read in data

### Load all datasets

In [98]:
#View the question data
question_data = pd.read_csv('data_all/question.csv')
print(question_data.shape)
question_data.head(5)


(129, 2)


,Column,QuestionText
0,Respondent,Randomized respondent ID number (not in order ...
1,Hobby,Do you code as a hobby?
2,OpenSource,Do you contribute to open source projects?
3,Country,In which country do you currently reside?
4,Student,"Are you currently enrolled in a formal, degree..."


In [99]:
#View the training data
train_data = pd.read_csv('data_all/train.csv')
#train_data.head(5)

In [100]:
#View the test data
test_data = pd.read_csv('data_all/test.csv')
#test_data['SalaryType'] = test_data['SalaryType'].fillna('Yearly')
#test_data.head(5)

In [101]:
#'Student','Employment', 'FormalEducation','UndergradMajor'
option_choices_cols_name= ['Hobby', 'OpenSource',
 'CompanySize', 'JobSatisfaction', 'CareerSatisfaction',
 'HopeFiveYears', 'JobSearchStatus', 'LastNewJob', 'UpdateCV', 'Currency', 'SalaryType',
 'TimeFullyProductive','AgreeDisagree1','AgreeDisagree2', 'AgreeDisagree3',
 'OperatingSystem','NumberMonitors', 'CheckInCode', 'AdBlocker','AdBlockerDisable','AdsAgreeDisagree1',
 'AdsAgreeDisagree2', 'AdsAgreeDisagree3', 'AIDangerous','AIInteresting',
 'AIResponsible', 'AIFuture','EthicsChoice', 'EthicsReport','EthicsResponsible','EthicalImplications',
 'StackOverflowRecommend','StackOverflowVisit','StackOverflowHasAccount', 'StackOverflowParticipate',
 'StackOverflowJobs','StackOverflowDevStory','StackOverflowJobsRecommend','StackOverflowConsiderMember', 'HypotheticalTools1',
 'HypotheticalTools2', 'HypotheticalTools3', 'HypotheticalTools4', 'HypotheticalTools5','WakeTime','HoursComputer',
 'HoursOutside', 'SkipMeals', 'Exercise', 'EducationParents','Age', 'Dependents',]
option_choices_cols_name

['Hobby',
 'OpenSource',
 'CompanySize',
 'JobSatisfaction',
 'CareerSatisfaction',
 'HopeFiveYears',
 'JobSearchStatus',
 'LastNewJob',
 'UpdateCV',
 'Currency',
 'SalaryType',
 'TimeFullyProductive',
 'AgreeDisagree1',
 'AgreeDisagree2',
 'AgreeDisagree3',
 'OperatingSystem',
 'NumberMonitors',
 'CheckInCode',
 'AdBlocker',
 'AdBlockerDisable',
 'AdsAgreeDisagree1',
 'AdsAgreeDisagree2',
 'AdsAgreeDisagree3',
 'AIDangerous',
 'AIInteresting',
 'AIResponsible',
 'AIFuture',
 'EthicsChoice',
 'EthicsReport',
 'EthicsResponsible',
 'EthicalImplications',
 'StackOverflowRecommend',
 'StackOverflowVisit',
 'StackOverflowHasAccount',
 'StackOverflowParticipate',
 'StackOverflowJobs',
 'StackOverflowDevStory',
 'StackOverflowJobsRecommend',
 'StackOverflowConsiderMember',
 'HypotheticalTools1',
 'HypotheticalTools2',
 'HypotheticalTools3',
 'HypotheticalTools4',
 'HypotheticalTools5',
 'WakeTime',
 'HoursComputer',
 'HoursOutside',
 'SkipMeals',
 'Exercise',
 'EducationParents',
 'Age',
 'D

# Generate and mapping features by country

In [102]:
#Generate some features by country
Gen_map_country=train_data[train_data['Salary']>0]
Gen_map_country['avg_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('mean')
Gen_map_country['min_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('min')
Gen_map_country['max_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform('max')
Gen_map_country['count_sal'] = Gen_map_country['No'].groupby(Gen_map_country['Country']).transform('count')
Gen_map_country['mode_sal'] = Gen_map_country['Salary'].groupby(Gen_map_country['Country']).transform(lambda x:x.value_counts().index[0])
Gen_map_country['std_sal']=Gen_map_country.Country.map(Gen_map_country.groupby('Country')['Salary'].std())

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [103]:
#Create map for country
Map_country=Gen_map_country[['Country','avg_sal','min_sal','max_sal','count_sal','mode_sal','std_sal']]
Salmap_country=Map_country.drop_duplicates()
#Salmap

In [104]:
#Mapping new gen features to train, test base on Country
train_data=pd.merge(train_data,Salmap_country,on='Country',how='left')
test_data=pd.merge(test_data,Salmap_country,on='Country',how='left')

# Generate and mapping features by YearsCoding

In [105]:
#Generate some features by YearsCoding
Gen_map_YearsCoding=train_data[train_data['Salary']>0]
Gen_map_YearsCoding['YearsCoding_avg_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('mean')
Gen_map_YearsCoding['YearsCoding_min_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('min')
Gen_map_YearsCoding['YearsCoding_max_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('max')
Gen_map_YearsCoding['YearsCoding_count_sal'] = Gen_map_YearsCoding['No'].groupby(Gen_map_YearsCoding['YearsCoding']).transform('count')
Gen_map_YearsCoding['YearsCoding_std_sal']=Gen_map_YearsCoding.YearsCoding.map(Gen_map_YearsCoding.groupby('YearsCoding')['Salary'].std())
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [106]:
#Create map for year coding
Map_Yearscoding=Gen_map_YearsCoding[['YearsCoding','YearsCoding_avg_sal','YearsCoding_min_sal',
                                     'YearsCoding_max_sal','YearsCoding_count_sal','YearsCoding_std_sal']]
Salmap_Yearscoding=Map_Yearscoding.drop_duplicates()
#Salmap

In [107]:
#Mapping new gen features to train, test base on Yearscoding
train_data=pd.merge(train_data,Salmap_Yearscoding,on='YearsCoding',how='left')
test_data=pd.merge(test_data,Salmap_Yearscoding,on='YearsCoding',how='left')

# Generate some features by RaceEthnicity

In [108]:
#Generate some features by RaceEthnicity
Gen_map_RaceEthnicity=train_data[train_data['Salary']>0]
Gen_map_RaceEthnicity['RaceEthnicity_avg_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('mean')
Gen_map_RaceEthnicity['RaceEthnicity_min_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('min')
Gen_map_RaceEthnicity['RaceEthnicity_max_sal'] = Gen_map_RaceEthnicity['Salary'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('max')
Gen_map_RaceEthnicity['RaceEthnicity_count_sal'] = Gen_map_RaceEthnicity['No'].groupby(Gen_map_RaceEthnicity['RaceEthnicity']).transform('count')
Gen_map_RaceEthnicity['RaceEthnicity_std_sal']=Gen_map_RaceEthnicity.RaceEthnicity.map(Gen_map_RaceEthnicity.groupby('RaceEthnicity')['Salary'].std())
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [109]:
#Create map for RaceEthnicity
Map_RaceEthnicity=Gen_map_RaceEthnicity[['RaceEthnicity','RaceEthnicity_avg_sal','RaceEthnicity_min_sal',
                                         'RaceEthnicity_max_sal','RaceEthnicity_count_sal','RaceEthnicity_std_sal']]
Salmap_RaceEthnicity=Map_RaceEthnicity.drop_duplicates()
#Salmap

In [110]:
#Mapping new gen features to train, test base on RaceEthnicity
train_data=pd.merge(train_data,Salmap_RaceEthnicity,on='RaceEthnicity',how='left')
test_data=pd.merge(test_data,Salmap_RaceEthnicity,on='RaceEthnicity',how='left')

# Generate some features by YearsCodingProf

In [111]:
#Generate some features by YearsCodingProf
Gen_map_YearsCodingProf=train_data[train_data['Salary']>0]
Gen_map_YearsCodingProf['YearsCodingProf_avg_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('mean')
Gen_map_YearsCodingProf['YearsCodingProf_min_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('min')
Gen_map_YearsCodingProf['YearsCodingProf_max_sal'] = Gen_map_YearsCodingProf['Salary'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('max')
Gen_map_YearsCodingProf['YearsCodingProf_std_sal']=Gen_map_YearsCodingProf.YearsCodingProf.map(Gen_map_YearsCodingProf.groupby('YearsCodingProf')['Salary'].std())
#Gen_map_YearsCodingProf['YearsCoding_count_sal'] = Gen_map_YearsCodingProf['No'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('count')
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [112]:
#Create map for YearsCodingProf
Map_YearsCodingProf=Gen_map_YearsCodingProf[['YearsCodingProf','YearsCodingProf_avg_sal','YearsCodingProf_min_sal',
                                     'YearsCodingProf_max_sal','YearsCodingProf_std_sal']]
Salmap_YearsCodingProf=Map_YearsCodingProf.drop_duplicates()
#Salmap
#Mapping new gen features to train, test base on YearsCodingProf
train_data=pd.merge(train_data,Salmap_YearsCodingProf,on='YearsCodingProf',how='left')
test_data=pd.merge(test_data,Salmap_YearsCodingProf,on='YearsCodingProf',how='left')

# Generate some features by CurrencySymbol

In [113]:
#Generate some features by CurrencySymbol
Gen_map_CurrencySymbol=train_data[train_data['Salary']>0]
Gen_map_CurrencySymbol['CurrencySymbol_avg_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('mean')
Gen_map_CurrencySymbol['CurrencySymbol_min_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('min')
Gen_map_CurrencySymbol['CurrencySymbol_max_sal'] = Gen_map_CurrencySymbol['Salary'].groupby(Gen_map_CurrencySymbol['CurrencySymbol']).transform('max')
Gen_map_CurrencySymbol['CurrencySymbol_std_sal']=Gen_map_CurrencySymbol.CurrencySymbol.map(Gen_map_CurrencySymbol.groupby('CurrencySymbol')['Salary'].std())
#Gen_map_YearsCodingProf['YearsCoding_count_sal'] = Gen_map_YearsCodingProf['No'].groupby(Gen_map_YearsCodingProf['YearsCodingProf']).transform('count')
#Gen_map_YearsCoding['mode_sal'] = Gen_map_YearsCoding['Salary'].groupby(Gen_map_YearsCoding['YearsCoding']).transform(lambda x:x.value_counts().index[0])

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCo

In [114]:
#Create map for YearsCodingProf
Map_CurrencySymbol=Gen_map_CurrencySymbol[['CurrencySymbol','CurrencySymbol_avg_sal','CurrencySymbol_min_sal',
                                     'CurrencySymbol_max_sal','CurrencySymbol_std_sal']]
Salmap_CurrencySymbol=Map_CurrencySymbol.drop_duplicates()
#Salmap
#Mapping new gen features to train, test base on YearsCodingProf
train_data=pd.merge(train_data,Salmap_CurrencySymbol,on='CurrencySymbol',how='left')
test_data=pd.merge(test_data,Salmap_CurrencySymbol,on='CurrencySymbol',how='left')

# Generate some features for all choice columns

In [115]:
for col in option_choices_cols_name:
    print('Start column: '+col)
    name='Gen_map_'+col
    avgname=col+'_avg_sal'
    minname=col+'_min_sal'
    maxname=col+'_max_sal'
    Mapname='Map_'+col
    Salmapname='Salmap_'+col
    name=train_data[train_data['Salary']>0]
    name[avgname] = name['Salary'].groupby(name[col]).transform('mean')
    name[minname] = name['Salary'].groupby(name[col]).transform('min')
    name[maxname] = name['Salary'].groupby(name[col]).transform('max')
    #Create map for YearsCodingProf
    Mapname=name[[col,avgname,minname,maxname]]
    Salmapname=Mapname.drop_duplicates()
    #Salmap
    #Mapping new gen features to train, test base on YearsCodingProf
    train_data=pd.merge(train_data,Salmapname,on=col,how='left')
    test_data=pd.merge(test_data,Salmapname,on=col,how='left')

Start column: Hobby


c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarni

Start column: OpenSource
Start column: CompanySize
Start column: JobSatisfaction
Start column: CareerSatisfaction
Start column: HopeFiveYears
Start column: JobSearchStatus
Start column: LastNewJob
Start column: UpdateCV
Start column: Currency
Start column: SalaryType
Start column: TimeFullyProductive
Start column: AgreeDisagree1
Start column: AgreeDisagree2
Start column: AgreeDisagree3
Start column: OperatingSystem
Start column: NumberMonitors
Start column: CheckInCode
Start column: AdBlocker
Start column: AdBlockerDisable
Start column: AdsAgreeDisagree1
Start column: AdsAgreeDisagree2
Start column: AdsAgreeDisagree3
Start column: AIDangerous
Start column: AIInteresting
Start column: AIResponsible
Start column: AIFuture
Start column: EthicsChoice
Start column: EthicsReport
Start column: EthicsResponsible
Start column: EthicalImplications
Start column: StackOverflowRecommend
Start column: StackOverflowVisit
Start column: StackOverflowHasAccount
Start column: StackOverflowParticipate
Sta

In [116]:
print(test_data.shape)
print(train_data.shape)

(11259, 307)
(33857, 308)


In [117]:
test_data.head()

,No,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,...,Exercise_max_sal,EducationParents_avg_sal,EducationParents_min_sal,EducationParents_max_sal,Age_avg_sal,Age_min_sal,Age_max_sal,Dependents_avg_sal,Dependents_min_sal,Dependents_max_sal
0,33858,No,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",NaN,Back-end developer;Front-end developer;Full-st...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,33859,Yes,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)",NaN,100 to 499 employees,Back-end developer;Database administrator;DevO...,...,260000.0,65552.154087,12.0,251750.0,57748.193844,1.0,260000.0,75873.884846,14.0,280000.0
2,33860,Yes,No,United States,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",20 to 99 employees,Front-end developer;Full-stack developer,...,250000.0,74732.295177,24.0,280000.0,33209.115588,12.0,200000.0,54663.863872,1.0,258000.0
3,33861,Yes,Yes,India,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Another engineering discipline (ex. civil, ele...",NaN,Back-end developer,...,260000.0,74732.295177,24.0,280000.0,33209.115588,12.0,200000.0,54663.863872,1.0,258000.0
4,33862,Yes,No,Norway,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",Fewer than 10 employees,Front-end developer;Full-stack developer;Mobil...,...,280000.0,65552.154087,12.0,251750.0,85614.294381,24.0,258000.0,75873.884846,14.0,280000.0


### Examine the distribution of target data

In [118]:
target = train_data['Salary'].astype(int)

### Examine the missing values on training data

In [119]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [120]:
missing_values = missing_values_table(train_data)
missing_values.head(10)

Your selected dataframe has 308 columns.
There are 293 columns that have missing values.


,Missing Values,% of Total Values
TimeAfterBootcamp,30638,90.5
MilitaryUS,25092,74.1
HackathonReasons,20939,61.8
ErgonomicDevices,18628,55.0
AdBlockerReasons,16431,48.5
StackOverflowJobsRecommend_avg_sal,15239,45.0
StackOverflowJobsRecommend_min_sal,15239,45.0
StackOverflowJobsRecommend_max_sal,15239,45.0
StackOverflowJobsRecommend,15239,45.0
JobEmailPriorities3,10841,32.0


In [121]:
missing_row_values = train_data.isnull().sum(axis=1)
missing_row_percentage = 100*missing_row_values/(train_data.shape[1]-1)

In [122]:
missing_row_df = pd.concat([missing_row_values,missing_row_percentage],axis=1)

In [123]:
missing_row_df_ren = missing_row_df.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})

In [124]:
missing_row_df_ren[missing_row_df_ren.iloc[:,1]!=0].sort_values(
        '% of Total Values', ascending=False).round(1).head(10)

,Missing Values,% of Total Values
15718,241,78.5
29860,239,77.9
23584,238,77.5
21537,236,76.9
15283,236,76.9
13451,235,76.5
30123,235,76.5
2173,234,76.2
23209,234,76.2
22301,234,76.2


## Column types

Convert some category columns to numeric

In [125]:
train_data["YearsCoding_convert"] = train_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCoding_convert"] = test_data["YearsCoding"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [126]:
train_data["YearsCodingProf_convert"] = train_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})
test_data["YearsCodingProf_convert"] = test_data["YearsCodingProf"].map( {'0-2 years': 1, '3-5 years': 4,'6-8 years': 7, 
                                                            '9-11 years': 10, '12-14 years': 13, '15-17 years': 16,
                                                           '18-20 years': 19, '21-23 years': 22, '24-26 years': 25,
                                                           '27-29 years': 28, '30 or more years': 30})

In [127]:
train_data["StackOverflowRecommend_convert"] = train_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowRecommend_convert"] = test_data["StackOverflowRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [128]:
train_data["StackOverflowJobsRecommend_convert"] = train_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})
test_data["StackOverflowJobsRecommend_convert"] = test_data["StackOverflowJobsRecommend"].map( {'10 (Very Likely)': 10,
                                                                                  '0 (Not Likely)': 0})

In [129]:
train_data["HoursComputer_convert"] = train_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})
test_data["HoursComputer_convert"] = test_data["HoursComputer"].map( {'1 - 4 hours': 3,'5 - 8 hours': 7,
                                                               '9 - 12 hours': 11,'Over 12 hours': 14})

In [130]:
train_data["HoursOutside_convert"] = train_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})
test_data["HoursOutside_convert"] = test_data["HoursOutside"].map( {'Less than 30 minutes': 0.3,'30 - 59 minutes': 0.8,
                                                               '1 - 2 hours': 1.5,'3 - 4 hours': 3.5, 'Over 4 hours': 5})

In [131]:
train_data["Age_convert"] = train_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})
test_data["Age_convert"] = test_data["Age"].map( {'18 - 24 years old': 21,'25 - 34 years old': 30,
                                                               '35 - 44 years old': 40,'45 - 54 years old': 50})

In [132]:
train_data["TimeFullyProductive_convert"] = train_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})
test_data["TimeFullyProductive_convert"] = test_data["TimeFullyProductive"].map( {'Less than a month': 1,'One to three months': 3,
                                                               'Six to nine months': 9,'Nine months to a year': 12,
                                                                           'More than a year': 15, 'Three to six months':6})

In [133]:
train_data["Hobby_convert"] = train_data["Hobby"].map( {'Yes': 1.0,'No': 0.0})
test_data["Hobby_convert"] = test_data["Hobby"].map( {'Yes': 1.0,'No': 0.0})

In [134]:
train_data["OpenSource_convert"] = train_data["OpenSource"].map( {'Yes': 1.0,'No': 0.0})
test_data["OpenSource_convert"] = test_data["OpenSource"].map( {'Yes': 1.0,'No': 0.0})

In [135]:
train_data["Student_convert"] = train_data["Student"].map( {'No': 2.0,'Yes, part-time': 1.0,'Yes, full-time': 0.0})
test_data["Student_convert"] = test_data["Student"].map( {'No': 2.0,'Yes, part-time': 1.0,'Yes, full-time': 0.0})

In [136]:
train_data["FormalEducation_convert"] = train_data["FormalEducation"].map( {'I never completed any formal education': 0.0,
            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1.0,
            'Primary/elementary school': 2.0,
            'Associate degree': 3.0,
            'Some college/university study without earning a degree': 4.0,
            'Bachelor’s degree (BA, BS, B.Eng., etc.)': 5.0,
            'Master’s degree (MA, MS, M.Eng., MBA, etc.)': 6.0,
            'Other doctoral degree (Ph.D, Ed.D., etc.)': 7.0,
            'Professional degree (JD, MD, etc.)': 8.0})
test_data["FormalEducation_convert"] = test_data["FormalEducation"].map( {'I never completed any formal education': 0.0,
            'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 1.0,
            'Primary/elementary school': 2.0,
            'Associate degree': 3.0,
            'Some college/university study without earning a degree': 4.0,
            'Bachelor’s degree (BA, BS, B.Eng., etc.)': 5.0,
            'Master’s degree (MA, MS, M.Eng., MBA, etc.)': 6.0,
            'Other doctoral degree (Ph.D, Ed.D., etc.)': 7.0,
            'Professional degree (JD, MD, etc.)': 8.0})

In [137]:
train_data["UndergradMajor_convert"] = train_data["UndergradMajor"].map({'Computer science, computer engineering, or software engineering': 2.0,
            'A natural science (ex. biology, chemistry, physics)': 1.0,
            'Web development or web design': 2.0,
            'Information systems, information technology, or system administration': 2.0,
            'A social science (ex. anthropology, psychology, political science)': 0.0,
            'A business discipline (ex. accounting, finance, marketing))': 0.0,
            'Another engineering discipline (ex. civil, electrical, mechanical)': 1.0,
            'A humanities discipline (ex. literature, history, philosophy)': 0.0,
            'Mathematics or statistics)': 2.0,
            'Fine arts or performing arts (ex. graphic design, music, studio art': 1.0,
            'I never declared a major': 0.0,
            'A health science (ex. nursing, pharmacy, radiology)': 0.0})
test_data["UndergradMajor_convert"] = test_data["UndergradMajor"].map({'Computer science, computer engineering, or software engineering': 2.0,
            'A natural science (ex. biology, chemistry, physics)': 1.0,
            'Web development or web design': 2.0,
            'Information systems, information technology, or system administration': 2.0,
            'A social science (ex. anthropology, psychology, political science)': 0.0,
            'A business discipline (ex. accounting, finance, marketing))': 0.0,
            'Another engineering discipline (ex. civil, electrical, mechanical)': 1.0,
            'A humanities discipline (ex. literature, history, philosophy)': 0.0,
            'Mathematics or statistics)': 2.0,
            'Fine arts or performing arts (ex. graphic design, music, studio art': 1.0,
            'I never declared a major': 0.0,
            'A health science (ex. nursing, pharmacy, radiology)': 0.0})

In [138]:
# Number of each type of column
train_data.dtypes.value_counts()

float64    234
object      86
int64        1
dtype: int64

### Examine OBJECT type columns

In [139]:
# Number of unique classes in each object column
train_data.select_dtypes('object').apply(pd.Series.nunique, axis = 0).sort_values(ascending=False).head(10)

LanguageDesireNextYear     14619
LanguageWorkedWith         13583
PlatformDesireNextYear      7501
DatabaseDesireNextYear      5658
PlatformWorkedWith          5249
DevType                     4884
IDE                         4623
DatabaseWorkedWith          4171
FrameworkDesireNextYear     1231
CommunicationTools           935
dtype: int64

In [140]:
#examine the few columns have most of classes
train_data[['LanguageDesireNextYear','LanguageWorkedWith','PlatformDesireNextYear']].head(10)

,LanguageDesireNextYear,LanguageWorkedWith,PlatformDesireNextYear
0,C#;Java;PHP;Python;SQL;HTML;CSS,PHP;SQL;HTML;CSS,Android;Linux;Windows Desktop or Server;WordPress
1,Go;Java;Kotlin,Go;Groovy;Java;JavaScript;Bash/Shell,Linux
2,Haskell,C++;JavaScript;SQL;HTML;CSS,Arduino;Linux;Raspberry Pi
3,Assembly;C;C++,Assembly;C;C++;Go;Python;Bash/Shell,Linux
4,C++;C#;Java;JavaScript;PHP;Python;SQL;HTML;CSS,C#;Java;JavaScript;Lua;PHP;SQL;VB.NET;HTML,Arduino;Gaming console;Linux;Mac OS;Windows De...
5,TypeScript,C#;JavaScript;SQL;HTML;CSS,Android;iOS
6,Rust;Kotlin;HTML;CSS,Java;JavaScript;Rust;Kotlin;HTML;CSS,Amazon Echo;Android;Arduino;AWS;iOS;Raspberry ...
7,Go;Java;JavaScript;PHP;SQL;HTML;CSS,Java;JavaScript;PHP;SQL;HTML;CSS;Bash/Shell,AWS;Serverless
8,JavaScript;PHP;SQL;Bash/Shell,PHP;SQL;Bash/Shell,Amazon Echo;AWS;Serverless
9,C#;JavaScript;PHP;SQL;TypeScript;HTML;CSS,JavaScript;PHP;SQL;HTML;CSS,NaN


### Examine FLOAT type columns

In [141]:
train_data.select_dtypes('float').head(10)

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert,Hobby_convert,OpenSource_convert,Student_convert,FormalEducation_convert,UndergradMajor_convert
0,9.0,7.0,8.0,3.0,6.0,10.0,2.0,1.0,5.0,4.0,...,NaN,7.0,5.0,40.0,NaN,1.0,0.0,2.0,5.0,2.0
1,9.0,2.0,1.0,6.0,4.0,8.0,3.0,5.0,10.0,7.0,...,NaN,14.0,0.8,40.0,15.0,1.0,1.0,2.0,6.0,1.0
2,9.0,4.0,6.0,1.0,3.0,5.0,2.0,8.0,10.0,7.0,...,NaN,7.0,1.5,40.0,3.0,1.0,1.0,2.0,4.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,7.0,0.8,30.0,3.0,1.0,0.0,2.0,5.0,2.0
4,3.0,8.0,10.0,2.0,7.0,9.0,1.0,5.0,6.0,4.0,...,NaN,14.0,0.3,NaN,NaN,0.0,1.0,2.0,2.0,NaN
5,9.0,1.0,8.0,5.0,4.0,6.0,10.0,3.0,7.0,2.0,...,10.0,11.0,0.8,30.0,3.0,1.0,0.0,2.0,5.0,2.0
6,8.0,10.0,9.0,4.0,6.0,2.0,7.0,1.0,3.0,5.0,...,10.0,11.0,0.3,30.0,1.0,1.0,1.0,2.0,4.0,2.0
7,10.0,6.0,4.0,7.0,5.0,1.0,9.0,2.0,8.0,3.0,...,NaN,11.0,1.5,30.0,3.0,1.0,1.0,2.0,6.0,2.0
8,10.0,1.0,9.0,5.0,2.0,3.0,6.0,4.0,7.0,8.0,...,NaN,11.0,0.3,30.0,3.0,1.0,1.0,2.0,5.0,2.0
9,9.0,8.0,7.0,6.0,2.0,1.0,5.0,4.0,10.0,3.0,...,NaN,11.0,0.3,21.0,6.0,1.0,1.0,2.0,4.0,2.0


In [142]:
view=train_data.select_dtypes('float').drop('Salary',axis=1).describe()
view

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert,Hobby_convert,OpenSource_convert,Student_convert,FormalEducation_convert,UndergradMajor_convert
count,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,32390.000000,...,5270.000000,32601.000000,32610.000000,31133.000000,28310.000000,33857.000000,33857.000000,33563.000000,33370.000000,27753.000000
mean,6.601019,6.753226,5.919759,4.027138,3.714078,4.302593,5.610157,4.232664,7.716764,6.122600,...,8.886148,10.112297,1.259212,31.070568,3.537337,0.812476,0.437221,1.698209,4.831885,1.741938
std,2.715798,2.498056,2.643799,2.508112,2.411668,2.453037,2.902357,2.476109,2.377179,2.625833,...,3.146384,2.447597,0.925838,7.441640,2.712045,0.390337,0.496050,0.671825,1.366173,0.559122
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,3.000000,0.300000,21.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.000000,5.000000,4.000000,2.000000,2.000000,2.000000,3.000000,2.000000,6.000000,4.000000,...,10.000000,7.000000,0.800000,30.000000,1.000000,1.000000,0.000000,2.000000,5.000000,2.000000
50%,7.000000,7.000000,6.000000,4.000000,3.000000,4.000000,6.000000,4.000000,8.000000,6.000000,...,10.000000,11.000000,0.800000,30.000000,3.000000,1.000000,0.000000,2.000000,5.000000,2.000000
75%,9.000000,9.000000,8.000000,6.000000,5.000000,6.000000,8.000000,6.000000,10.000000,8.000000,...,10.000000,11.000000,1.500000,30.000000,6.000000,1.000000,1.000000,2.000000,6.000000,2.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,14.000000,5.000000,50.000000,15.000000,1.000000,1.000000,2.000000,8.000000,2.000000


### Examine INT type columns

In [143]:
train_data.select_dtypes('int').head(10)

""
0
1
2
3
4
5
6
7
8
9


In [144]:
# drop int column type
train_data = train_data.drop('No',axis=1)
test_data = test_data.drop('No',axis=1)

# Data preprocessing

## Clean data

Drop the columns and the rows have more 50% missing values

In [145]:
missing_cols_drop = missing_values[missing_values.iloc[:,1]>80].index

In [146]:
train_data = train_data.drop(missing_cols_drop,axis =1)
test_data = test_data.drop(missing_cols_drop,axis =1)

In [147]:
missing_rows_drop = train_data.isnull().sum(axis=1)

In [148]:
train_data = train_data[missing_rows_drop<(train_data.shape[1]/2)]

In [149]:
print('Training data shape:',train_data.shape)
print('Testing data shape:',test_data.shape)

Training data shape: (33076, 319)
Testing data shape: (11259, 318)


In [150]:
target = train_data['Salary']
train_data = train_data.drop('Salary',axis=1)

## Split dataset to 3 small datasets based on column type

### Float columns type

In [151]:
# get columns name
float_cols_name_train = train_data.select_dtypes('float').columns
float_cols_name_test = test_data.select_dtypes('float').columns

In [152]:
#fill na by median
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_arr = imputer.fit_transform(train_data.select_dtypes('float'))
test_arr = imputer.transform(test_data.select_dtypes('float'))

In [153]:
float_dataset_train = pd.DataFrame(train_arr,columns=float_cols_name_train)
float_dataset_test = pd.DataFrame(test_arr,columns=float_cols_name_test)

In [154]:
print('Training float dataset shape: ', float_dataset_train.shape)
print('Testing dloat dataset shape: ', float_dataset_test.shape)

Training float dataset shape:  (33076, 233)
Testing dloat dataset shape:  (11259, 233)


In [155]:
float_dataset_train.head()

,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,...,StackOverflowJobsRecommend_convert,HoursComputer_convert,HoursOutside_convert,Age_convert,TimeFullyProductive_convert,Hobby_convert,OpenSource_convert,Student_convert,FormalEducation_convert,UndergradMajor_convert
0,9.0,7.0,8.0,3.0,6.0,10.0,2.0,1.0,5.0,4.0,...,10.0,7.0,5.0,40.0,3.0,1.0,0.0,2.0,5.0,2.0
1,9.0,2.0,1.0,6.0,4.0,8.0,3.0,5.0,10.0,7.0,...,10.0,14.0,0.8,40.0,15.0,1.0,1.0,2.0,6.0,1.0
2,9.0,4.0,6.0,1.0,3.0,5.0,2.0,8.0,10.0,7.0,...,10.0,7.0,1.5,40.0,3.0,1.0,1.0,2.0,4.0,2.0
3,7.0,7.0,6.0,4.0,3.0,4.0,6.0,4.0,8.0,6.0,...,10.0,7.0,0.8,30.0,3.0,1.0,0.0,2.0,5.0,2.0
4,3.0,8.0,10.0,2.0,7.0,9.0,1.0,5.0,6.0,4.0,...,10.0,14.0,0.3,30.0,3.0,0.0,1.0,2.0,2.0,2.0


In [156]:
#Check corr
#Checkcorr= float_dataset_train
#Checkcorr['label']=target

In [157]:
# The undersample dataframe
#plt.figure(figsize = (20, 20))
#corr = float_dataset_train.corr()
#sns.heatmap(corr, cmap="coolwarm_r", annot=True)

In [158]:
#corr['label']

### Categorical "multiple-choices" columns type

In [159]:
#function for check which column contains multiple-choice
def check_multiple_choices(text):
    if type(text)==str:
        if ';' in text:
            return 1
        else:
            return 0
    return 0

In [160]:
#Categorical columns is columns, which are not in float cols
categorical_cols_name=[col for col in train_data.columns if col not in float_cols_name_train]
#Multiple choices is categorical with character: ";"
mul_choices_cols_name=[]
for i in categorical_cols_name:
    if sum(train_data[i].apply(check_multiple_choices))>0:
        mul_choices_cols_name.append(i)
option_choices_cols_name=[col for col in categorical_cols_name if col not in mul_choices_cols_name]

In [161]:
#list of columns have multiple-choice on train_dataset
mul_choices_cols_name

['DevType',
 'CommunicationTools',
 'EducationTypes',
 'SelfTaughtTypes',
 'HackathonReasons',
 'LanguageWorkedWith',
 'LanguageDesireNextYear',
 'DatabaseWorkedWith',
 'DatabaseDesireNextYear',
 'PlatformWorkedWith',
 'PlatformDesireNextYear',
 'FrameworkWorkedWith',
 'FrameworkDesireNextYear',
 'IDE',
 'Methodology',
 'VersionControl',
 'AdBlockerReasons',
 'AdsActions',
 'ErgonomicDevices',
 'Gender',
 'SexualOrientation',
 'RaceEthnicity']

In [162]:
#function for count the number of choices on each column
def count_multiple_choice(text):
    if type(text)==str:
        return text.split(";")
    else:
        return []

In [163]:
num_multiple_choice={}
for i in mul_choices_cols_name:
    flat_ls = list(train_data[i].apply(count_multiple_choice))
    num_multiple_choice[i]=len(set(list(itertools.chain(*flat_ls))))

In [164]:
#display the number of choice on columns
#frame_num_choice = pd.DataFrame(num_multiple_choice.items(),columns=['Feature names','Number of choice']).sort_values('Number of choice',ascending=False)
#frame_num_choice

#### Encode multiple-choices to one-hot

In [165]:
#function for encoding multiple-choice to one hot
def encode(ls,ref):
    ini_vec = np.zeros((len(ref)))
    ini_ls=[]
    for i in ls:
        ini_ls.append(ref.index(i))
    ini_vec[ini_ls]=1
    return ini_vec.astype('int')

In [166]:
#apply the function to handle multiple-choices columns
ls_v4_train=[]
ls_v4_test=[]
ls_v2=[]
for c in mul_choices_cols_name:
    
    v1_train = train_data[c].apply(count_multiple_choice) #series
    v1_test = test_data[c].apply(count_multiple_choice) 
    
    v2 = list(set(list(itertools.chain(*v1_train)))) #list of unique of multiple-choices
    v3_train = v1_train.apply(encode,ref=v2) #numpy array
    v3_test = v1_test.apply(encode,ref=v2)
    
    v4_train = np.concatenate(v3_train.values).reshape(-1,len(v2))
    v4_test = np.concatenate(v3_test.values).reshape(-1,len(v2))
    ls_v4_train.append(v4_train)
    ls_v4_test.append(v4_test)
    ls_v2.append(v2)
v5_train = np.concatenate(ls_v4_train,axis=1)
v5_test = np.concatenate(ls_v4_test,axis=1)
v6 = list(itertools.chain(*ls_v2))
multi_choice_dataset_train = pd.DataFrame(v5_train,columns = v6)
multi_choice_dataset_test = pd.DataFrame(v5_test,columns = v6)

In [167]:
multi_choice_dataset_train.head()

,Engineering manager,Front-end developer,System administrator,QA or test developer,Product manager,Designer,Marketing or sales professional,"C-suite executive (CEO, CTO, etc.)",Full-stack developer,Game or graphics developer,...,Straight or heterosexual,Asexual,Gay or Lesbian,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,South Asian,East Asian,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian"
0,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0


In [168]:
multi_choice_dataset_test.head()

,Engineering manager,Front-end developer,System administrator,QA or test developer,Product manager,Designer,Marketing or sales professional,"C-suite executive (CEO, CTO, etc.)",Full-stack developer,Game or graphics developer,...,Straight or heterosexual,Asexual,Gay or Lesbian,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,South Asian,East Asian,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian"
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,1,0,0,0,0,0


In [169]:
cols=pd.Series(multi_choice_dataset_train.columns)
for dup in cols[cols.duplicated()].unique(): 
    cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
multi_choice_dataset_train.columns=cols
multi_choice_dataset_test.columns=cols

In [170]:
test_data['DevType'].head(5)

0    Back-end developer;Front-end developer;Full-st...
1    Back-end developer;Database administrator;DevO...
2             Front-end developer;Full-stack developer
3                                   Back-end developer
4    Front-end developer;Full-stack developer;Mobil...
Name: DevType, dtype: object

In [171]:
multi_choice_dataset_train.iloc[:5,:20]

,Engineering manager,Front-end developer,System administrator,QA or test developer,Product manager,Designer,Marketing or sales professional,"C-suite executive (CEO, CTO, etc.)",Full-stack developer,Game or graphics developer,Student,DevOps specialist,Database administrator,Data or business analyst,Desktop or enterprise applications developer,Embedded applications or devices developer,Data scientist or machine learning specialist,Mobile developer,Educator or academic researcher,Back-end developer
0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1


In [172]:
multi_choice_dataset_train_generate= multi_choice_dataset_train[cols]
multi_choice_dataset_train_generate['Salary']=target
multi_choice_dataset_train_generate.head()

,Engineering manager,Front-end developer,System administrator,QA or test developer,Product manager,Designer,Marketing or sales professional,"C-suite executive (CEO, CTO, etc.)",Full-stack developer,Game or graphics developer,...,Asexual,Gay or Lesbian,Hispanic or Latino/Latina,White or of European descent,Black or of African descent,South Asian,East Asian,Middle Eastern,"Native American, Pacific Islander, or Indigenous Australian",Salary
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,10800.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,73433.0
2,0,0,0,0,0,1,0,0,1,0,...,0,0,0,1,0,0,0,0,0,66672.0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,31848.0
4,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0.0


In [173]:
for col in multi_choice_dataset_train.columns:
    print('Start column: '+col)
    name='Gen_map_'+col
    avgname=col+'_avg_sal'
    minname=col+'_min_sal'
    maxname=col+'_max_sal'
    Mapname='Map_'+col
    Salmapname='Salmap_'+col
    name=multi_choice_dataset_train_generate[multi_choice_dataset_train_generate['Salary']>0]
    name[avgname] = name['Salary'].groupby(name[col]).transform('mean')
    name[minname] = name['Salary'].groupby(name[col]).transform('min')
    name[maxname] = name['Salary'].groupby(name[col]).transform('max')
    #Create map for YearsCodingProf
    Mapname=name[[col,avgname,minname,maxname]]
    Salmapname=Mapname.drop_duplicates()
    #print(Salmapname)
    #Mapping new gen features to train, test base on YearsCodingProf
    multi_choice_dataset_train=pd.merge(multi_choice_dataset_train,Salmapname,on=col,how='left')
    multi_choice_dataset_test=pd.merge(multi_choice_dataset_test,Salmapname,on=col,how='left')

Start column: Engineering manager


c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarni

Start column: Front-end developer
Start column: System administrator
Start column: QA or test developer
Start column: Product manager
Start column: Designer
Start column: Marketing or sales professional
Start column: C-suite executive (CEO, CTO, etc.)
Start column: Full-stack developer
Start column: Game or graphics developer
Start column: Student
Start column: DevOps specialist
Start column: Database administrator
Start column: Data or business analyst
Start column: Desktop or enterprise applications developer
Start column: Embedded applications or devices developer
Start column: Data scientist or machine learning specialist
Start column: Mobile developer
Start column: Educator or academic researcher
Start column: Back-end developer
Start column: Jira
Start column: Slack
Start column: HipChat
Start column: Other wiki tool (Github, Google Sites, proprietary software, etc.)
Start column: Office / productivity suite (Microsoft Office, Google Suite, etc.)
Start column: Stack Overflow Ente

Start column: Xcode
Start column: NetBeans
Start column: Notepad++
Start column: Atom
Start column: Android Studio
Start column: PHPStorm
Start column: Mob programming
Start column: Kanban
Start column: Lean
Start column: Extreme programming (XP)
Start column: Evidence-based software engineering
Start column: PRINCE2
Start column: Pair programming
Start column: Formal standard such as ISO 9001 or IEEE 12207 (aka “waterfall” methodologies)
Start column: Agile
Start column: Scrum
Start column: Zip file back-ups
Start column: Mercurial
Start column: Copying and pasting files to network shares
Start column: I don't use version control
Start column: Team Foundation Version Control
Start column: Subversion
Start column: Git
Start column: The ad-blocking software was causing display issues on a website
Start column: The website I was visiting forced me to disable it to access their content
Start column: The website I was visiting asked me to disable it
Start column: I wanted to view a specifi

In [174]:
#multi_choice_dataset_train= multi_choice_dataset_train.drop('Salary',axis=1)
print('Training multiple-choices dataset shape: ', multi_choice_dataset_train.shape)
print('Testing multiple-choices dataset shape: ', multi_choice_dataset_test.shape)

Training multiple-choices dataset shape:  (33076, 1280)
Testing multiple-choices dataset shape:  (11259, 1280)


### Categorical "option-choices" columns type

In [175]:
option_choices_cols_name=[col for col in categorical_cols_name if col not in mul_choices_cols_name]

In [176]:
option_choices_cols_name

['Hobby',
 'OpenSource',
 'Country',
 'Student',
 'Employment',
 'FormalEducation',
 'UndergradMajor',
 'CompanySize',
 'YearsCoding',
 'YearsCodingProf',
 'JobSatisfaction',
 'CareerSatisfaction',
 'HopeFiveYears',
 'JobSearchStatus',
 'LastNewJob',
 'UpdateCV',
 'Currency',
 'SalaryType',
 'CurrencySymbol',
 'TimeFullyProductive',
 'AgreeDisagree1',
 'AgreeDisagree2',
 'AgreeDisagree3',
 'OperatingSystem',
 'NumberMonitors',
 'CheckInCode',
 'AdBlocker',
 'AdBlockerDisable',
 'AdsAgreeDisagree1',
 'AdsAgreeDisagree2',
 'AdsAgreeDisagree3',
 'AIDangerous',
 'AIInteresting',
 'AIResponsible',
 'AIFuture',
 'EthicsChoice',
 'EthicsReport',
 'EthicsResponsible',
 'EthicalImplications',
 'StackOverflowRecommend',
 'StackOverflowVisit',
 'StackOverflowHasAccount',
 'StackOverflowParticipate',
 'StackOverflowJobs',
 'StackOverflowDevStory',
 'StackOverflowJobsRecommend',
 'StackOverflowConsiderMember',
 'HypotheticalTools1',
 'HypotheticalTools2',
 'HypotheticalTools3',
 'HypotheticalTools4

In [177]:
#impute missing value by majority class
imputer = SimpleImputer(missing_values=np.nan,strategy='most_frequent')
train_data_arr = imputer.fit_transform(train_data[option_choices_cols_name])
test_data_arr = imputer.transform(test_data[option_choices_cols_name])

In [178]:
option_choices_dataset_train = pd.DataFrame(train_data_arr,columns=option_choices_cols_name)
option_choices_dataset_test = pd.DataFrame(test_data_arr,columns=option_choices_cols_name)

#### Encode option-choices to one-hot columns

In [179]:
# Create a label encoder for option columns
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in option_choices_cols_name:
    # If 2 or fewer unique categories
    if len(list(option_choices_dataset_train[col].unique())) <= 2:
        # Train on the training data
        le.fit(option_choices_dataset_train[col])
        # Transform both training and testing data
        option_choices_dataset_train[col] = le.transform(option_choices_dataset_train[col])
        option_choices_dataset_test[col] = le.transform(option_choices_dataset_test[col])    
        # Keep track of how many columns were label encoded
        le_count += 1
            
print('%d columns were label encoded.' % le_count)

4 columns were label encoded.


In [180]:
#one-hot encode for train_data
option_choices_dataset_train = pd.get_dummies(option_choices_dataset_train)
option_choices_dataset_train.shape

(33076, 587)

In [181]:
#one-hot encode for test_data
option_choices_dataset_test = pd.get_dummies(option_choices_dataset_test)
option_choices_dataset_test.shape

(11259, 547)

In [182]:
#Align the training and testing data, keep only columns present in both dataframes
option_choices_dataset_train, option_choices_dataset_test = option_choices_dataset_train.align(option_choices_dataset_test, join = 'inner', axis = 1)
print('Training option-choices dataset shape: ', option_choices_dataset_train.shape)
print('Testing option-choices dataset shape: ', option_choices_dataset_test.shape)

Training option-choices dataset shape:  (33076, 537)
Testing option-choices dataset shape:  (11259, 537)


### Concate all dataframe

In [183]:
#float_dataset_train,
train_dataset = pd.concat([float_dataset_train,multi_choice_dataset_train,option_choices_dataset_train],axis=1)

In [184]:
#float_dataset_test,
test_dataset = pd.concat([float_dataset_test,multi_choice_dataset_test,option_choices_dataset_test],axis=1)

In [185]:
for col in train_dataset.columns:
    print (col)
    namecol='squared_of_'+col
    train_dataset[namecol]=train_dataset[col]**2

AssessJob1
AssessJob2
AssessJob3
AssessJob4
AssessJob5
AssessJob6
AssessJob7
AssessJob8
AssessJob9
AssessJob10
AssessBenefits1
AssessBenefits2
AssessBenefits3
AssessBenefits4
AssessBenefits5
AssessBenefits6
AssessBenefits7
AssessBenefits8
AssessBenefits9
AssessBenefits10
AssessBenefits11
JobContactPriorities1
JobContactPriorities2
JobContactPriorities3
JobContactPriorities4
JobContactPriorities5
JobEmailPriorities1
JobEmailPriorities2
JobEmailPriorities3
JobEmailPriorities4
JobEmailPriorities5
JobEmailPriorities6
JobEmailPriorities7
AdsPriorities1
AdsPriorities2
AdsPriorities3
AdsPriorities4
AdsPriorities5
AdsPriorities6
AdsPriorities7
avg_sal
min_sal
max_sal
count_sal
mode_sal
std_sal
YearsCoding_avg_sal
YearsCoding_min_sal
YearsCoding_max_sal
YearsCoding_count_sal
YearsCoding_std_sal
RaceEthnicity_avg_sal
RaceEthnicity_min_sal
RaceEthnicity_max_sal
RaceEthnicity_count_sal
RaceEthnicity_std_sal
YearsCodingProf_avg_sal
YearsCodingProf_min_sal
YearsCodingProf_max_sal
YearsCodingProf_std

Amazon RDS/Aurora_1
Google BigQuery_1
Amazon Redshift_1
Cassandra_1
IBM Db2_1
Memcached_1
SQLite_1
Neo4j_1
Amazon DynamoDB_1
Apache Hive_1
Elasticsearch_1
MongoDB_1
Redis_1
Oracle_1
Apache HBase_1
MySQL_1
Linux
ESP8266
Salesforce
Mainframe
IBM Cloud or Watson
Predix
Apple Watch or Apple TV
Arduino
Windows Desktop or Server
Windows Phone
Azure
Serverless
WordPress
Mac OS
Heroku
ESP 8266
iOS
Drupal
AWS
Google Cloud Platform/App Engine
Amazon Echo
Google Home
SharePoint
Gaming console
Android
Firebase
Raspberry Pi
Linux_1
ESP8266_1
Salesforce_1
Mainframe_1
IBM Cloud or Watson_1
Predix_1
Apple Watch or Apple TV_1
Arduino_1
Windows Desktop or Server_1
Windows Phone_1
Azure_1
Serverless_1
WordPress_1
Mac OS_1
Heroku_1
ESP 8266_1
iOS_1
Drupal_1
AWS_1
Google Cloud Platform/App Engine_1
Amazon Echo_1
Google Home_1
SharePoint_1
Gaming console_1
Android_1
Firebase_1
Raspberry Pi_1
Spark
Torch/PyTorch
Xamarin
Node.js
.NET Core
Angular
React
Hadoop
Spring
Cordova
Django
TensorFlow
Spark_1
Torch/PyT

The technology’s online help system_max_sal
A college/university computer science or software engineering book_avg_sal
A college/university computer science or software engineering book_min_sal
A college/university computer science or software engineering book_max_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_avg_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_min_sal
Pre-scheduled tutoring or mentoring sessions with a friend or colleague_max_sal
Internal Wikis, chat rooms, or documentation set up by my company for employees_avg_sal
Internal Wikis, chat rooms, or documentation set up by my company for employees_min_sal
Internal Wikis, chat rooms, or documentation set up by my company for employees_max_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_avg_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_min_sal
A book or e-book from O’Reilly, Apress, or a similar publisher_max_sal
Tapping your n

Linux_min_sal
Linux_max_sal
ESP8266_avg_sal
ESP8266_min_sal
ESP8266_max_sal
Salesforce_avg_sal
Salesforce_min_sal
Salesforce_max_sal
Mainframe_avg_sal
Mainframe_min_sal
Mainframe_max_sal
IBM Cloud or Watson_avg_sal
IBM Cloud or Watson_min_sal
IBM Cloud or Watson_max_sal
Predix_avg_sal
Predix_min_sal
Predix_max_sal
Apple Watch or Apple TV_avg_sal
Apple Watch or Apple TV_min_sal
Apple Watch or Apple TV_max_sal
Arduino_avg_sal
Arduino_min_sal
Arduino_max_sal
Windows Desktop or Server_avg_sal
Windows Desktop or Server_min_sal
Windows Desktop or Server_max_sal
Windows Phone_avg_sal
Windows Phone_min_sal
Windows Phone_max_sal
Azure_avg_sal
Azure_min_sal
Azure_max_sal
Serverless_avg_sal
Serverless_min_sal
Serverless_max_sal
WordPress_avg_sal
WordPress_min_sal
WordPress_max_sal
Mac OS_avg_sal
Mac OS_min_sal
Mac OS_max_sal
Heroku_avg_sal
Heroku_min_sal
Heroku_max_sal
ESP 8266_avg_sal
ESP 8266_min_sal
ESP 8266_max_sal
iOS_avg_sal
iOS_min_sal
iOS_max_sal
Drupal_avg_sal
Drupal_min_sal
Drupal_max_s

Male_avg_sal
Male_min_sal
Male_max_sal
Female_avg_sal
Female_min_sal
Female_max_sal
Transgender_avg_sal
Transgender_min_sal
Transgender_max_sal
Non-binary, genderqueer, or gender non-conforming_avg_sal
Non-binary, genderqueer, or gender non-conforming_min_sal
Non-binary, genderqueer, or gender non-conforming_max_sal
Bisexual or Queer_avg_sal
Bisexual or Queer_min_sal
Bisexual or Queer_max_sal
Straight or heterosexual_avg_sal
Straight or heterosexual_min_sal
Straight or heterosexual_max_sal
Asexual_avg_sal
Asexual_min_sal
Asexual_max_sal
Gay or Lesbian_avg_sal
Gay or Lesbian_min_sal
Gay or Lesbian_max_sal
Hispanic or Latino/Latina_avg_sal
Hispanic or Latino/Latina_min_sal
Hispanic or Latino/Latina_max_sal
White or of European descent_avg_sal
White or of European descent_min_sal
White or of European descent_max_sal
Black or of African descent_avg_sal
Black or of African descent_min_sal
Black or of African descent_max_sal
South Asian_avg_sal
South Asian_min_sal
South Asian_max_sal
East As

CurrencySymbol_ETB
CurrencySymbol_EUR
CurrencySymbol_GBP
CurrencySymbol_GEL
CurrencySymbol_GHS
CurrencySymbol_HKD
CurrencySymbol_HRK
CurrencySymbol_HUF
CurrencySymbol_IDR
CurrencySymbol_ILS
CurrencySymbol_INR
CurrencySymbol_IRR
CurrencySymbol_ISK
CurrencySymbol_JOD
CurrencySymbol_JPY
CurrencySymbol_KES
CurrencySymbol_KRW
CurrencySymbol_KWD
CurrencySymbol_KZT
CurrencySymbol_LKR
CurrencySymbol_MAD
CurrencySymbol_MGA
CurrencySymbol_MKD
CurrencySymbol_MNT
CurrencySymbol_MUR
CurrencySymbol_MXN
CurrencySymbol_MYR
CurrencySymbol_MZN
CurrencySymbol_NGN
CurrencySymbol_NOK
CurrencySymbol_NPR
CurrencySymbol_NZD
CurrencySymbol_PEN
CurrencySymbol_PHP
CurrencySymbol_PKR
CurrencySymbol_PLN
CurrencySymbol_PYG
CurrencySymbol_RON
CurrencySymbol_RSD
CurrencySymbol_RUB
CurrencySymbol_SAR
CurrencySymbol_SDG
CurrencySymbol_SEK
CurrencySymbol_SGD
CurrencySymbol_SYP
CurrencySymbol_THB
CurrencySymbol_TND
CurrencySymbol_TRY
CurrencySymbol_TTD
CurrencySymbol_TWD
CurrencySymbol_TZS
CurrencySymbol_UAH
CurrencySymb

In [186]:
for col in test_dataset.columns:
    print (col)
    namecol='squared_of_'+col
    test_dataset[namecol]=test_dataset[col]**2

AssessJob1
AssessJob2
AssessJob3
AssessJob4
AssessJob5
AssessJob6
AssessJob7
AssessJob8
AssessJob9
AssessJob10
AssessBenefits1
AssessBenefits2
AssessBenefits3
AssessBenefits4
AssessBenefits5
AssessBenefits6
AssessBenefits7
AssessBenefits8
AssessBenefits9
AssessBenefits10
AssessBenefits11
JobContactPriorities1
JobContactPriorities2
JobContactPriorities3
JobContactPriorities4
JobContactPriorities5
JobEmailPriorities1
JobEmailPriorities2
JobEmailPriorities3
JobEmailPriorities4
JobEmailPriorities5
JobEmailPriorities6
JobEmailPriorities7
AdsPriorities1
AdsPriorities2
AdsPriorities3
AdsPriorities4
AdsPriorities5
AdsPriorities6
AdsPriorities7
avg_sal
min_sal
max_sal
count_sal
mode_sal
std_sal
YearsCoding_avg_sal
YearsCoding_min_sal
YearsCoding_max_sal
YearsCoding_count_sal
YearsCoding_std_sal
RaceEthnicity_avg_sal
RaceEthnicity_min_sal
RaceEthnicity_max_sal
RaceEthnicity_count_sal
RaceEthnicity_std_sal
YearsCodingProf_avg_sal
YearsCodingProf_min_sal
YearsCodingProf_max_sal
YearsCodingProf_std

Mac OS_1
Heroku_1
ESP 8266_1
iOS_1
Drupal_1
AWS_1
Google Cloud Platform/App Engine_1
Amazon Echo_1
Google Home_1
SharePoint_1
Gaming console_1
Android_1
Firebase_1
Raspberry Pi_1
Spark
Torch/PyTorch
Xamarin
Node.js
.NET Core
Angular
React
Hadoop
Spring
Cordova
Django
TensorFlow
Spark_1
Torch/PyTorch_1
Xamarin_1
Node.js_1
.NET Core_1
Angular_1
React_1
Hadoop_1
Spring_1
Cordova_1
Django_1
TensorFlow_1
RStudio
Emacs
IntelliJ
Sublime Text
PyCharm
Zend
Coda
Komodo
TextMate
IPython / Jupyter
Visual Studio Code
Eclipse
Vim
RubyMine
Light Table
Visual Studio
Xcode
NetBeans
Notepad++
Atom
Android Studio
PHPStorm
Mob programming
Kanban
Lean
Extreme programming (XP)
Evidence-based software engineering
PRINCE2
Pair programming
Formal standard such as ISO 9001 or IEEE 12207 (aka “waterfall” methodologies)
Agile
Scrum
Zip file back-ups
Mercurial
Copying and pasting files to network shares
I don't use version control
Team Foundation Version Control
Subversion
Git
The ad-blocking software was causing 

TypeScript_min_sal
TypeScript_max_sal
Java_avg_sal
Java_min_sal
Java_max_sal
Bash/Shell_avg_sal
Bash/Shell_min_sal
Bash/Shell_max_sal
JavaScript_avg_sal
JavaScript_min_sal
JavaScript_max_sal
C_avg_sal
C_min_sal
C_max_sal
Lua_avg_sal
Lua_min_sal
Lua_max_sal
Go_avg_sal
Go_min_sal
Go_max_sal
Cobol_avg_sal
Cobol_min_sal
Cobol_max_sal
Matlab_avg_sal
Matlab_min_sal
Matlab_max_sal
R_avg_sal
R_min_sal
R_max_sal
Perl_avg_sal
Perl_min_sal
Perl_max_sal
PHP_avg_sal
PHP_min_sal
PHP_max_sal
C#_avg_sal
C#_min_sal
C#_max_sal
Hack_avg_sal
Hack_min_sal
Hack_max_sal
Swift_avg_sal
Swift_min_sal
Swift_max_sal
Ruby_avg_sal
Ruby_min_sal
Ruby_max_sal
Kotlin_avg_sal
Kotlin_min_sal
Kotlin_max_sal
SQL_avg_sal
SQL_min_sal
SQL_max_sal
Objective-C_1_avg_sal
Objective-C_1_min_sal
Objective-C_1_max_sal
Scala_1_avg_sal
Scala_1_min_sal
Scala_1_max_sal
HTML_1_avg_sal
HTML_1_min_sal
HTML_1_max_sal
Assembly_1_avg_sal
Assembly_1_min_sal
Assembly_1_max_sal
Groovy_1_avg_sal
Groovy_1_min_sal
Groovy_1_max_sal
CoffeeScript_1_av

TensorFlow_max_sal
Spark_1_avg_sal
Spark_1_min_sal
Spark_1_max_sal
Torch/PyTorch_1_avg_sal
Torch/PyTorch_1_min_sal
Torch/PyTorch_1_max_sal
Xamarin_1_avg_sal
Xamarin_1_min_sal
Xamarin_1_max_sal
Node.js_1_avg_sal
Node.js_1_min_sal
Node.js_1_max_sal
.NET Core_1_avg_sal
.NET Core_1_min_sal
.NET Core_1_max_sal
Angular_1_avg_sal
Angular_1_min_sal
Angular_1_max_sal
React_1_avg_sal
React_1_min_sal
React_1_max_sal
Hadoop_1_avg_sal
Hadoop_1_min_sal
Hadoop_1_max_sal
Spring_1_avg_sal
Spring_1_min_sal
Spring_1_max_sal
Cordova_1_avg_sal
Cordova_1_min_sal
Cordova_1_max_sal
Django_1_avg_sal
Django_1_min_sal
Django_1_max_sal
TensorFlow_1_avg_sal
TensorFlow_1_min_sal
TensorFlow_1_max_sal
RStudio_avg_sal
RStudio_min_sal
RStudio_max_sal
Emacs_avg_sal
Emacs_min_sal
Emacs_max_sal
IntelliJ_avg_sal
IntelliJ_min_sal
IntelliJ_max_sal
Sublime Text_avg_sal
Sublime Text_min_sal
Sublime Text_max_sal
PyCharm_avg_sal
PyCharm_min_sal
PyCharm_max_sal
Zend_avg_sal
Zend_min_sal
Zend_max_sal
Coda_avg_sal
Coda_min_sal
Coda

JobSatisfaction_Moderately dissatisfied
JobSatisfaction_Moderately satisfied
JobSatisfaction_Neither satisfied nor dissatisfied
JobSatisfaction_Slightly dissatisfied
JobSatisfaction_Slightly satisfied
CareerSatisfaction_Extremely dissatisfied
CareerSatisfaction_Extremely satisfied
CareerSatisfaction_Moderately dissatisfied
CareerSatisfaction_Moderately satisfied
CareerSatisfaction_Neither satisfied nor dissatisfied
CareerSatisfaction_Slightly dissatisfied
CareerSatisfaction_Slightly satisfied
HopeFiveYears_Doing the same work
HopeFiveYears_Retirement
HopeFiveYears_Working as a founder or co-founder of my own company
HopeFiveYears_Working as a product manager or project manager
HopeFiveYears_Working as an engineering manager or other functional manager
HopeFiveYears_Working in a career completely unrelated to software development
HopeFiveYears_Working in a different or more specialized technical role than the one I'm in now
JobSearchStatus_I am actively looking for a job
JobSearchStatus

HypotheticalTools1_A little bit interested
HypotheticalTools1_Extremely interested
HypotheticalTools1_Not at all interested
HypotheticalTools1_Somewhat interested
HypotheticalTools1_Very interested
HypotheticalTools2_A little bit interested
HypotheticalTools2_Extremely interested
HypotheticalTools2_Not at all interested
HypotheticalTools2_Somewhat interested
HypotheticalTools2_Very interested
HypotheticalTools3_A little bit interested
HypotheticalTools3_Extremely interested
HypotheticalTools3_Not at all interested
HypotheticalTools3_Somewhat interested
HypotheticalTools3_Very interested
HypotheticalTools4_A little bit interested
HypotheticalTools4_Extremely interested
HypotheticalTools4_Not at all interested
HypotheticalTools4_Somewhat interested
HypotheticalTools4_Very interested
HypotheticalTools5_A little bit interested
HypotheticalTools5_Extremely interested
HypotheticalTools5_Not at all interested
HypotheticalTools5_Somewhat interested
HypotheticalTools5_Very interested
WakeTime_A

In [187]:
print('Training dataset shape: ', train_dataset.shape)
#print('Training label shape: ', y_train.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (33076, 4100)
Testing dataset shape:  (11259, 4100)


In [188]:
#1106

# 2. TRAINING

In [189]:
from sklearn.utils.validation import column_or_1d

In [190]:
X_train= train_dataset
y_train= target.astype(int)
X_test= test_dataset

In [191]:
y_train.dtype

dtype('int32')

In [192]:
import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
X_test = X_test.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

### Split training set to validation set

In [193]:
params = {
    'objective'         : 'regression',    
    'metric'            : 'l2', 
    'nthread'           : 4,
    'learning_rate'     : 0.001,

    'num_leaves'        : 23,
    'feature_fraction'  : 0.106,
    'bagging_fraction'  : 0.825,
    'max_depth'         : -1,
    'lambda_l1'         : 0.1,
    'lambda_l2'         : 2.5,
    'min_split_gain'    : 0.007,
}

# # Create parameters to search
# gridParams = {
#     'lambda_l1' : [0.1,0.2,0.3],
#     'lambda_l2' : [2.5,2.7,2.9],
#     }

# # Create REGRESSOR to use. Note that parameters have to be input manually
# # not as a dict!
# mdl = lgb.LGBMRegressor(
#           objective = params['objective'],
#           metric = params['metric'],
#           nthread = params['nthread'],
#           learning_rate = params['learning_rate'],
#           num_leaves = params['num_leaves'],
#           feature_fraction = params['feature_fraction'],
#           bagging_fraction = params['bagging_fraction'],
#           max_depth = params['max_depth'],
#           lambda_l1 = params['lambda_l1'],
#           lambda_l2 = params['lambda_l2'],
#           min_split_gain = params['min_split_gain'])

# # To view the default model params:
# mdl.get_params().keys()

# # Create the grid
# grid = GridSearchCV(mdl, gridParams,
#                     verbose=0,
#                     cv=4,
#                     n_jobs=2)

# # Run the grid
# grid.fit(X_train, y_train)

# # Print the best parameters found
# print(grid.best_params_)
# print(grid.best_score_)

# # Using parameters already set above, replace in the best from the grid search
# params['lambda_l1'] = grid.best_params_['lambda_l1']
# params['lambda_l2'] = grid.best_params_['lambda_l2']

# print('Fitting with params: ')
# print(params)

In [194]:
skf = StratifiedKFold(n_splits=4, random_state=3462873, shuffle=True)
preds = 0.0
for itrain, ivalid in skf.split(X_train, y_train):
    lgb_train = lgb.Dataset(X_train.iloc[itrain], y_train.iloc[itrain])
    lgb_eval  = lgb.Dataset(X_train.iloc[ivalid], y_train.iloc[ivalid], reference = lgb_train)
    model = lgb.train(params,
                lgb_train,
                num_boost_round = 999999,  
                early_stopping_rounds = 800,
                verbose_eval = False,
                valid_sets = [lgb_train, lgb_eval])
    pred = model.predict(X_test)
    preds += pred/skf.n_splits

c:\users\gobiz\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


# 3. PREDICTING

In [195]:
def to_submit_file(file_name, probs):
    """
    A helper function helps generating submit files
    
    :param file_name: submit file name
    :param probs: list of probabilities for test samples
    
    :return: None
    """
    submit = test_id.copy()
    submit['Salary'] = probs
    submit.to_csv(file_name, index = False)
    print ('Done writting submit file: ', file_name)

In [196]:
test_id = pd.read_csv('data_all/submit.csv')

In [197]:
y_pred = preds

In [198]:
to_submit_file('submitfile.csv',y_pred)

Done writting submit file:  submitfile.csv
